In [5]:
import pandas as pd

In [6]:
spam = pd.read_csv("spam.csv")

In [8]:
spam.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
spam.columns = ['label','text']

In [11]:
spam.head()


,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
import spacy

# Create an empty model
nlp = spacy.blank("en")

In [18]:
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL
print(DEFAULT_SINGLE_TEXTCAT_MODEL)

{'@architectures': 'spacy.TextCatEnsemble.v2', 'linear_model': {'@architectures': 'spacy.TextCatBOW.v1', 'exclusive_classes': True, 'ngram_size': 1, 'no_output_layer': False}, 'tok2vec': {'@architectures': 'spacy.Tok2Vec.v2', 'embed': {'@architectures': 'spacy.MultiHashEmbed.v2', 'width': 64, 'rows': [2000, 2000, 1000, 1000, 1000, 1000], 'attrs': ['ORTH', 'LOWER', 'PREFIX', 'SUFFIX', 'SHAPE', 'ID'], 'include_static_vectors': False}, 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2', 'width': 64, 'window_size': 1, 'maxout_pieces': 3, 'depth': 2}}}


In [19]:
config={"model":{"@architectures" : "spacy.TextCatBOW.v1",
"exclusive_classes" : True,
"ngram_size" : 1,
"no_output_layer" :False},}
textcat = nlp.add_pipe("textcat",config=config)

# textcat = nlp.add_pipe("textcat")
print(textcat.cfg)

{'labels': [], 'threshold': 0.5, 'positive_label': None}


In [20]:
textcat.add_label("ham")
textcat.add_label("spam")

1

In [22]:

train_texts = spam['text'].values
train_labels = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}} 
                for label in spam['label']]
#print(train_labels)

In [23]:

train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}})]

In [24]:
import random
from spacy.util import minibatch
# V3
from spacy.tokens import Doc
from spacy.training import Example
# --------


spacy.util.fix_random_seed(1)
nlp.begin_training()
optimizer = nlp.initialize()
losses={}

for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=200)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        # v2
        # texts, labels = zip(*batch)
        # nlp.update(texts, labels, sgd=optimizer)
        # V3-------------------------------------
        examples = [Example.from_dict(nlp(text), cats) for text, cats in train_data]
        nlp.update(examples, sgd=optimizer, losses=losses)
        print(losses)

# v3 -----------------------------
# Training data for text categorization
# doc = nlp("I'm pretty happy about that!")
# gold_dict = {"cats": {"POSITIVE": 1.0, "NEGATIVE": 0.0}}
# example = Example.from_dict(doc, gold_dict)

{'textcat': 1.6104520383919407e-08}
{'textcat': 3.130481118773787e-08}
{'textcat': 4.5658095082501404e-08}
{'textcat': 5.922299806115916e-08}
{'textcat': 7.205672325483192e-08}
{'textcat': 8.421390340629387e-08}
{'textcat': 9.574599069139822e-08}
{'textcat': 1.0670087746689205e-07}
{'textcat': 1.1712271152930498e-07}
{'textcat': 1.2705185614692027e-07}
{'textcat': 1.365249859830442e-07}
{'textcat': 1.4557529404157776e-07}
{'textcat': 1.5423277321957585e-07}
{'textcat': 1.625245351632998e-07}
{'textcat': 1.7047513267698378e-07}
{'textcat': 1.781068688089249e-07}
{'textcat': 1.8544007973630983e-07}
{'textcat': 1.9249338167881547e-07}
{'textcat': 1.9928388583778656e-07}
{'textcat': 2.0582737825236563e-07}
{'textcat': 2.1213847301027045e-07}
{'textcat': 2.1823073881321875e-07}
{'textcat': 2.2411680244971421e-07}
{'textcat': 2.2980843450426391e-07}
{'textcat': 2.3531662174391954e-07}
{'textcat': 2.4065162440578547e-07}
{'textcat': 2.4582302904363473e-07}
{'textcat': 2.5083979116047317e-07}


In [25]:

texts = ["Are you ready for the tea party????? It's gonna be wild",
         "URGENT Reply to this message for GUARANTEED FREE TEA" ]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

print(scores)

[[0.99220604 0.00779398]
 [0.28505814 0.71494186]]
